In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
import time
import selenium
import pyautogui

In [2]:
#파이썬 코드로 제어할 수 있는 네이버맵 창이 열림
wd = webdriver.Chrome()
url = "https://map.naver.com/"
wd.get(url) 

In [3]:
# 검색창에 버스번호를 치고 엔터 클릭됨
searchbox=wd.find_element(By.XPATH,"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div/div/input")
searchbox.send_keys("36")
searchbox.send_keys(Keys.ENTER)
time.sleep(2)

In [4]:
# 목록 중에서 특정 단어가 들어가면 바로 클릭하는 로직
bus_list = []
css_selector = ""
html = wd.page_source
soup = BeautifulSoup(html, "html.parser")
bus_lists = soup.find_all(name="div",attrs={"class":"search_result_content_area"}) # 페이지에 몇 개의 리스트가 있는지
len(bus_lists)
for i in range(len(bus_lists)):
    #bus_list.append(soup.find(name="div",attrs={"class":"search_result_content_area"}))
    bus_list.append(soup.select(f"#container > shrinkable-layout > div > app-base > search-layout > div.main.-top_space.ng-star-inserted > combined-search-list > fusion-search-list > fusion-bus-route-list > div > fusion-bus-route-item:nth-child({i+1})"))
    if '부산' in str(bus_list[i]):
        css_selector = f"#container > shrinkable-layout > div > app-base > search-layout > div.main.-top_space.ng-star-inserted > combined-search-list > fusion-search-list > fusion-bus-route-list > div > fusion-bus-route-item:nth-child({i+1})"
        break

elem = wd.find_element(By.CSS_SELECTOR, css_selector)
elem.click()
#container > shrinkable-layout > div > app-base > search-layout > div.main.-top_space.ng-star-inserted > combined-search-list > fusion-search-list > fusion-bus-route-list > div > fusion-bus-route-item:nth-child(2)
#container > shrinkable-layout > div > app-base > search-layout > div.main.-top_space.ng-star-inserted > combined-search-list > fusion-search-list > fusion-bus-route-list > div > fusion-bus-route-item:nth-child({i+1}) > div
#container > shrinkable-layout > div > app-base > search-layout > div.main.-top_space.ng-star-inserted > combined-search-list > fusion-search-list > fusion-bus-route-list > div > fusion-bus-route-item:nth-child(3)
#container > shrinkable-layout > div > app-base > search-layout > div.main.-top_space.ng-star-inserted > combined-search-list > fusion-search-list > fusion-bus-route-list > div > fusion-bus-route-item:nth-child(4)

In [5]:
from selenium.webdriver import ActionChains
html = wd.page_source
some_tag = wd.find_element(By.CLASS_NAME, 'suggest_area')

# somthing element 까지 스크롤
for i in range(5):
    action = (ActionChains(wd).move_to_element(some_tag)).perform()
    time.sleep(0.5)


In [6]:
# 클릭해서 새로운 화면이 나와서 다시 주소를 받아야 함
# 현재 클릭한 버스의 모든 정류장이 나옴
element = {} # 정류장과 정류장번호
memory = ""
x_path = ""
state = "LAST"
html = wd.page_source
soup = BeautifulSoup(html, "html.parser")
bus_stop_ids = soup.find_all(name="span",attrs={"class":"route_id"}) # 정류장 번호

for i in range (len(bus_stop_ids)):
    try:
        element[(wd.find_element(By.XPATH,f"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-bus-route/div[1]/div[3]/div/div[1]/div[2]/ul/li[{i+1}]/a/span[2]/strong")).text]= bus_stop_ids[i].text
        memory = f"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-bus-route/div[1]/div[3]/div/div[1]/div[2]/ul/li[{i+1}]/a/span[2]/strong"
    except:
        try:
            element[(wd.find_element(By.XPATH,f"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-bus-route/div[1]/div[2]/div/div[1]/div[2]/ul/li[{i+1}]/a/span[2]/strong")).text]= bus_stop_ids[i].text
            memory = f"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-bus-route/div[1]/div[2]/div/div[1]/div[2]/ul/li[{i+1}]/a/span[2]/strong"
        except:
            print("실패")
        
    if (wd.find_element(By.XPATH, memory).text == "안평.고촌"):
        if state == "LAST":
            state = "FIRST"
            continue
        time.sleep(1)
        button = wd.find_element(By.XPATH, memory)
        ActionChains(wd).click(button).perform()
        break;

완료


In [8]:
html = wd.page_source
soup = BeautifulSoup(html, "html.parser")
bus_times = wd.find_element(By.XPATH, "/html/body/div[1]/div[1]/div/div[2]/div/div[3]/div[3]/div/div/div[1]/div/div/div[2]/div")
bus_times = bus_times.text.replace("저장","").replace(" ","").replace("출발도착", "").split()
cur_bus_times = []
for i in range(len(bus_times)):
    cur_bus_times.append(bus_times[i].replace("착", "착 ").replace("장","장 / ").replace("분","분 "))
    
cur_bus_times
#(((bus_times.text.replace(" ","")).split()))
 #replace("저장"," ")).replace("출발도착","")).split())

['36',
 '17분 15정류장 / 45분 35정류장 / ',
 '73',
 '운행정보없음',
 '73(아침)',
 '55분 38정류장 / ',
 '183',
 '13분 9정류장 / 23분 18정류장 / ',
 '183(심야)',
 '운행정보없음']

In [6]:
pyautogui.moveTo(700, 550)

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
import time
import selenium
import pyautogui

#파이썬 코드로 제어할 수 있는 네이버맵 창이 열림
wd = webdriver.Chrome()
url = "https://map.naver.com/"
wd.get(url) 
time.sleep(5)

# 검색창에 버스번호를 치고 엔터 클릭됨
searchbox=wd.find_element(By.XPATH,"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div/div/input")
searchbox.send_keys("36")
searchbox.send_keys(Keys.ENTER)
time.sleep(2)

# 목록 중에서 특정 단어가 들어가면 바로 클릭하는 로직
bus_list = []
css_selector = ""
html = wd.page_source
soup = BeautifulSoup(html, "html.parser")
bus_lists = soup.find_all(name="div",attrs={"class":"search_result_content_area"}) # 페이지에 몇 개의 리스트가 있는지
len(bus_lists)
for i in range(len(bus_lists)):
    #bus_list.append(soup.find(name="div",attrs={"class":"search_result_content_area"}))
    bus_list.append(soup.select(f"#container > shrinkable-layout > div > app-base > search-layout > div.main.-top_space.ng-star-inserted > combined-search-list > fusion-search-list > fusion-bus-route-list > div > fusion-bus-route-item:nth-child({i+1})"))
    if '부산' in str(bus_list[i]):
        css_selector = f"#container > shrinkable-layout > div > app-base > search-layout > div.main.-top_space.ng-star-inserted > combined-search-list > fusion-search-list > fusion-bus-route-list > div > fusion-bus-route-item:nth-child({i+1})"
        break
elem = wd.find_element(By.CSS_SELECTOR, css_selector)
elem.click()


while True:
    time.sleep(1)
    some_tag = wd.find_element(By.CLASS_NAME, 'suggest_area')

    # somthing element 까지 스크롤
    for i in range(5):
        action = (ActionChains(wd).move_to_element(some_tag)).perform()
        time.sleep(0.5)
    
# 클릭해서 새로운 화면이 나와서 다시 주소를 받아야 함
# 현재 클릭한 버스의 모든 정류장이 나옴
    element = {} # 정류장과 정류장번호
    memory = ""
    x_path = ""
    state = "FIRST"
    html = wd.page_source
    soup = BeautifulSoup(html, "html.parser")
    bus_stop_ids = soup.find_all(name="span",attrs={"class":"route_id"}) # 정류장 번호

    for i in range (len(bus_stop_ids)):
        try:
            element[(wd.find_element(By.XPATH,f"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-bus-route/div[1]/div[3]/div/div[1]/div[2]/ul/li[{i+1}]/a/span[2]/strong")).text]= bus_stop_ids[i].text
            memory = f"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-bus-route/div[1]/div[3]/div/div[1]/div[2]/ul/li[{i+1}]/a/span[2]/strong"
        except:
            try:
                element[(wd.find_element(By.XPATH,f"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-bus-route/div[1]/div[2]/div/div[1]/div[2]/ul/li[{i+1}]/a/span[2]/strong")).text]= bus_stop_ids[i].text
                memory = f"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-bus-route/div[1]/div[2]/div/div[1]/div[2]/ul/li[{i+1}]/a/span[2]/strong"
            except:
                print("실패")
        
        if (wd.find_element(By.XPATH, memory).text == "안평.고촌"):
            if state == "LAST":
                state = "FIRST"
                continue
            time.sleep(1)
            button = wd.find_element(By.XPATH, memory)
            ActionChains(wd).click(button).perform()
            break;

    html = wd.page_source
    soup = BeautifulSoup(html, "html.parser")
    bus_times = wd.find_element(By.XPATH, "/html/body/div[1]/div[1]/div/div[2]/div/div[3]/div[3]/div/div/div[1]/div/div/div[2]/div")
    bus_times = bus_times.text.replace("저장","").replace(" ","").replace("출발도착", "").split()
    cur_bus_times = []
    for i in range(len(bus_times)):
        cur_bus_times.append(bus_times[i].replace("착", "착 ").replace("장","장 / ").replace("분","분 "))
    print(cur_bus_times)
    wd.refresh()
    time.sleep(5)
    

['36', '6분 5정류장 / 24분 19정류장 / ', '73', '46분 34정류장 / ', '73(아침)', '운행정보없음', '183', '10분 8정류장 / 31분 27정류장 / ', '183(심야)', '운행정보없음']
['36', '6분 5정류장 / 23분 18정류장 / ', '73', '46분 34정류장 / ', '73(아침)', '운행정보없음', '183', '9분 8정류장 / 30분 27정류장 / ', '183(심야)', '운행정보없음']
['36', '6분 5정류장 / 23분 18정류장 / ', '73', '46분 34정류장 / ', '73(아침)', '운행정보없음', '183', '9분 8정류장 / 30분 27정류장 / ', '183(심야)', '운행정보없음']
['36', '6분 5정류장 / 23분 18정류장 / ', '73', '45분 33정류장 / ', '73(아침)', '운행정보없음', '183', '9분 7정류장 / 29분 25정류장 / ', '183(심야)', '운행정보없음']
['36', '6분 5정류장 / 23분 18정류장 / ', '73', '45분 33정류장 / ', '73(아침)', '운행정보없음', '183', '9분 7정류장 / 29분 25정류장 / ', '183(심야)', '운행정보없음']
['36', '6분 5정류장 / 23분 18정류장 / ', '73', '44분 33정류장 / ', '73(아침)', '운행정보없음', '183', '8분 7정류장 / 29분 25정류장 / ', '183(심야)', '운행정보없음']
['36', '6분 5정류장 / 23분 18정류장 / ', '73', '44분 33정류장 / ', '73(아침)', '운행정보없음', '183', '8분 7정류장 / 29분 25정류장 / ', '183(심야)', '운행정보없음']
['36', '5분 5정류장 / 23분 18정류장 / ', '73', '44분 33정류장 / ', '73(아침)', '운행정보없음', '183', '8분 6정류장 / 29분

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".suggest_area"}
  (Session info: chrome=114.0.5735.248); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B7A813+48355]
	(No symbol) [0x00B0C4B1]
	(No symbol) [0x00A15358]
	(No symbol) [0x00A409A5]
	(No symbol) [0x00A40B3B]
	(No symbol) [0x00A6E232]
	(No symbol) [0x00A5A784]
	(No symbol) [0x00A6C922]
	(No symbol) [0x00A5A536]
	(No symbol) [0x00A382DC]
	(No symbol) [0x00A393DD]
	GetHandleVerifier [0x00DDAABD+2539405]
	GetHandleVerifier [0x00E1A78F+2800735]
	GetHandleVerifier [0x00E1456C+2775612]
	GetHandleVerifier [0x00C051E0+616112]
	(No symbol) [0x00B15F8C]
	(No symbol) [0x00B12328]
	(No symbol) [0x00B1240B]
	(No symbol) [0x00B04FF7]
	BaseThreadInitThunk [0x768800C9+25]
	RtlGetAppContainerNamedObjectPath [0x77387B1E+286]
	RtlGetAppCo